In [1]:
import datetime
import os
import random
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import tensorflow as tf
import mlflow.keras
import importlib
import joblib
from sklearn.metrics import roc_auc_score, f1_score

import ml_investing_wne.config as config
from ml_investing_wne.data_engineering.load_data import get_hist_data
from ml_investing_wne.data_engineering.prepare_dataset import prepare_processed_dataset
from ml_investing_wne.train_test_val_split import train_test_val_split
from ml_investing_wne.helper import confusion_matrix_plot, compute_profitability_classes
from ml_investing_wne.utils import get_logger

seed = 12345
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
                   
logger = get_logger()

2022-10-23 12:09:53.434981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 12:09:53.435023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = get_hist_data(currency=config.currency)
df = prepare_processed_dataset(df=df)
X, y, X_val, y_val, X_test, y_test, y_cat, y_val_cat, y_test_cat, train = train_test_val_split(df)


2022-10-23 12:10:25,638 - ml_investing_wne.data_engineering.prepare_dataset - exported to /home/jupyter/ml_investing_wne/src/ml_investing_wne/data/processed/EURCHF/EURCHF_processed_240min.csv
2022-10-23 12:10:25,677 - ml_investing_wne.train_test_val_split - first sequence begins: 2010-01-11 00:00:00
2022-10-23 12:10:25,678 - ml_investing_wne.train_test_val_split - first sequence ends: 2010-02-01 08:00:00
2022-10-23 12:10:25,750 - ml_investing_wne.train_test_val_split - last sequence begins: 2019-12-09 00:00:00
2022-10-23 12:10:25,751 - ml_investing_wne.train_test_val_split - last sequence ends: 2019-12-30 20:00:00
2022-10-23 12:10:25,916 - ml_investing_wne.train_test_val_split - first sequence begins: 2019-12-15 16:00:00
2022-10-23 12:10:25,917 - ml_investing_wne.train_test_val_split - first sequence ends: 2020-01-07 08:00:00
2022-10-23 12:10:25,925 - ml_investing_wne.train_test_val_split - last sequence begins: 2020-12-08 20:00:00
2022-10-23 12:10:25,926 - ml_investing_wne.train_test_

In [3]:
model = load_model(os.path.join(config.package_directory, 'models', 'production',
                        '{}_{}_{}_{}_{}_{}'.format(config.model, 'hist_data',
                                                   config.currency, config.freq,
                                                   str(config.steps_ahead),
                                                   config.seq_len)))

2022-10-23 12:10:26.090318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-23 12:10:26.090365: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-23 12:10:26.090387: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (python-20220811): /proc/driver/nvidia/version does not exist
2022-10-23 12:10:26.090645: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
y_pred = model.predict(X_test)
y_pred_class = y_pred.argmax(axis=-1)

if 'JPY' in config.currency:
    df['cost'] = (config.pips / 100) / df['close']
else:
    df['cost'] = (config.pips / 10000) / df['close']

start_date = joblib.load(os.path.join(config.package_directory, 'models',
                                      'first_sequence_ends_{}_{}_{}.save'.format('test',
                                                                                 config.currency,
                                                                                 config.freq)))
end_date = joblib.load(os.path.join(config.package_directory, 'models',
                                    'last_sequence_ends_{}_{}_{}.save'.format('test',
                                                                              config.currency,
                                                                              config.freq)))
lower_bounds = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
upper_bounds = [1 - lower for lower in lower_bounds]

for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
    portfolio_result, hit_ratio, time_active = compute_profitability_classes(df, y_pred, start_date,
                                                                             end_date, lower_bound,
                                                                             upper_bound)

2022-10-23 12:10:28.295222: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
/home/jupyter/ml_investing_wne/src/ml_investing_wne/helper.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  'Europe/London').dt.tz_localize(None)
/home/jupyter/ml_investing_wne/src/ml_investing_wne/helper.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['hour_local'] = prediction['datetime_local'].dt.time
/home/jupyter/ml_investing_wne

In [5]:
check = df.copy()
check['datetime_local'] = check['datetime'].dt.tz_localize('US/Eastern').dt.tz_convert(
        'Europe/London').dt.tz_localize(None)
check['hour_local'] = check['datetime_local'].dt.time
check

,datetime,open,high,low,close,y_pred,SMA_3,EMA_3,VAR_3,SMA_5,...,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,cost,datetime_local,hour_local
0,2010-01-11 00:00:00,1.47410,1.47940,1.47350,1.47700,-0.000203,1.475500,1.475963,2.410000e-06,1.475360,...,1.002103,0,0,0.000000,1.000000,0.000000,1.0,0.000135,2010-01-11 05:00:00,05:00:00
1,2010-01-11 04:00:00,1.47710,1.47800,1.47510,1.47670,-0.000609,1.475867,1.476331,2.923333e-06,1.475680,...,0.999797,4,0,0.951057,0.309017,0.000000,1.0,0.000135,2010-01-11 09:00:00,09:00:00
2,2010-01-11 08:00:00,1.47660,1.47690,1.47420,1.47580,-0.000678,1.476500,1.476066,3.900000e-07,1.475800,...,0.999391,8,0,0.587785,-0.809017,0.000000,1.0,0.000136,2010-01-11 13:00:00,13:00:00
3,2010-01-11 12:00:00,1.47590,1.47630,1.47450,1.47480,-0.000203,1.475767,1.475433,9.033333e-07,1.475640,...,0.999322,12,0,-0.587785,-0.809017,0.000000,1.0,0.000136,2010-01-11 17:00:00,17:00:00
4,2010-01-11 16:00:00,1.47480,1.47530,1.47410,1.47450,0.000407,1.475033,1.474966,4.633333e-07,1.475760,...,0.999797,16,0,-0.951057,0.309017,0.000000,1.0,0.000136,2010-01-11 21:00:00,21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20285,2022-08-31 00:00:00,0.97607,0.97711,0.97378,0.97652,0.003512,0.976287,0.976142,5.083333e-08,0.976094,...,1.000461,0,2,0.000000,1.000000,0.866025,-0.5,0.000205,2022-08-31 05:00:00,05:00:00
20286,2022-08-31 04:00:00,0.97650,0.98065,0.97472,0.97995,0.001133,0.977513,0.978046,4.503633e-06,0.976830,...,1.003512,4,2,0.951057,0.309017,0.866025,-0.5,0.000204,2022-08-31 09:00:00,09:00:00
20287,2022-08-31 08:00:00,0.97993,0.98238,0.97802,0.98106,0.001468,0.979177,0.979553,5.601433e-06,0.977974,...,1.001133,8,2,0.587785,-0.809017,0.866025,-0.5,0.000204,2022-08-31 13:00:00,13:00:00
20288,2022-08-31 12:00:00,0.98107,0.98287,0.97925,0.98250,0.000458,0.981170,0.981027,1.634700e-06,0.979220,...,1.001468,12,2,-0.587785,-0.809017,0.866025,-0.5,0.000204,2022-08-31 17:00:00,17:00:00


In [6]:
lower_bounds = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
upper_bounds = [1 - lower for lower in lower_bounds]
# hours_to_exclude = [datetime.time(21,0,0), datetime.time(22,0,0),  datetime.time(23,0,0)] # 60  - wrong
# hours_to_exclude = [datetime.time(20,0,0), datetime.time(21,0,0),  datetime.time(22,0,0)] # 60 - correct, we have to consider that trade is opened at the end of interval for the next hour
hours_to_exclude = [datetime.time(19,0,0), datetime.time(21,0,0)] # 120 min


for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
    portfolio_result, hit_ratio, time_active = compute_profitability_classes(df, y_pred, start_date,
                                                                             end_date, lower_bound,
                                                                             upper_bound, hours_exclude=hours_to_exclude)

/home/jupyter/ml_investing_wne/src/ml_investing_wne/helper.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  'Europe/London').dt.tz_localize(None)
/home/jupyter/ml_investing_wne/src/ml_investing_wne/helper.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['hour_local'] = prediction['datetime_local'].dt.time
/home/jupyter/ml_investing_wne/src/ml_investing_wne/helper.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [21]:
pd.options.display.max_rows = None

In [12]:

import datetime

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def compute_profitability_classes(df, y_pred, date_start, date_end, lower_bound, upper_bound, hours_exclude=None):
    
    prediction = df.copy()
    prediction.reset_index(inplace=True)
    df['y_pred'] = df['close'].shift(-config.steps_ahead) / df['close'] - 1
    # new_start = config.val_end + config.seq_len * datetime.timedelta(minutes=int(''.join(filter(str.isdigit, config.freq))))
    prediction = df.loc[(df.datetime >= date_start) & (df.datetime <= date_end)]
    prediction['datetime_local'] = prediction['datetime'].dt.tz_localize('US/Eastern').dt.tz_convert(
        'Europe/London').dt.tz_localize(None)
    prediction['hour_local'] = prediction['datetime_local'].dt.time
    # prediction['trade'] = y_pred.argmax(axis=1)
    prediction['prediction'] = y_pred[:, 1]
    conditions = [
        (prediction['prediction'] <= lower_bound),
        (prediction['prediction'] > lower_bound) & (prediction['prediction'] <= upper_bound),
        (prediction['prediction'] > upper_bound)
    ]
    values = [0, 0.5, 1]
    prediction['trade'] = np.select(conditions, values)
    if hours_exclude:
        prediction.loc[prediction['hour_local'].isin(hours_exclude), 'trade'] = 0.5
    prediction.reset_index(inplace=True)
    budget = 100
    transaction = None
    i = 0
    # drop last row for which we don't have a label
    prediction.drop(prediction.tail(1).index, inplace=True)
    while i < prediction.shape[0]:
        # for i in range(prediction.shape[0]):
        if prediction.loc[i, 'trade'] == config.nb_classes - 1:
            # add transaction cost if position changes
            if transaction != 'buy':
                budget = budget * (1 - prediction.loc[i, 'cost'])
            transaction = 'buy'
            budget = budget + budget * prediction.loc[i, 'y_pred']
            prediction.loc[i, 'budget'] = budget
            prediction.loc[i, 'transaction'] = transaction
            i = i + config.steps_ahead
        elif prediction.loc[i, 'trade'] == 0:
            # add transaction cost if position changes
            if transaction != 'sell':
                budget = budget * (1 - prediction.loc[i, 'cost'])
            transaction = 'sell'
            budget = budget + budget * (-prediction.loc[i, 'y_pred'])
            prediction.loc[i, 'budget'] = budget
            prediction.loc[i, 'transaction'] = transaction
            i = i + config.steps_ahead
        elif prediction.loc[i, 'trade'] == 0.5:
            if transaction in ['buy', 'sell']:
                # budget = budget * (1 - prediction.loc[i, 'cost']) # spread is included once in transaction costs
                transaction = None
            prediction.loc[i, 'budget'] = budget
            prediction.loc[i, 'transaction'] = transaction
            i = i + 1

    hits = prediction.loc[((prediction['transaction'] == 'buy') & (prediction['y_pred'] > 0)) |
                          ((prediction['transaction'] == 'sell') & (prediction['y_pred'] < 0))].shape[0]
    transactions = prediction.loc[prediction['transaction'].isin(['buy', 'sell'])].shape[0]
    try:
        hits_ratio = hits / transactions
    except ZeroDivisionError:
        hits_ratio = 0
    share_of_time_active = round(prediction.loc[prediction['transaction'].isin(['buy', 'sell'])].shape[0] * \
                                 config.steps_ahead / prediction.shape[0], 2)
    logger.info('share_of_time_active for bounds {}-{} is {} and hit ratio is {}'.format(lower_bound, upper_bound,
                                                                                         share_of_time_active,
                                                                                         hits_ratio))
    plt.figure(2)
    plt.plot(prediction['datetime'], prediction['budget'])
    plt.axhline(y=100, color='r', linestyle='-')
    plt.savefig(os.path.join(config.package_directory, 'models', 'portfolio_evolution_{}_{}_{}_{}_{}.png'.
                             format(config.model, config.currency, config.nb_classes, lower_bound, upper_bound)))
    plt.close()

    logger.info('Portfolio result:  {}'.format(budget))
    
    return budget, hits_ratio, share_of_time_active, prediction



In [ ]:
portfolio_result, hit_ratio, time_active, prediction = compute_profitability_classes(df, y_pred, start_date,
                                                                             end_date, 0.45,
                                                                             0.55, hours_exclude=hours_to_exclude)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [17]:
pd.options.display.max_columns = None

In [18]:
prediction

,index,datetime,open,high,low,close,y_pred,SMA_3,EMA_3,VAR_3,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_13,EMA_13,VAR_13,SMA_20,EMA_20,VAR_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos,cost,datetime_local,hour_local,prediction,trade,budget,transaction
0,68362,2020-12-31 16:00:00,1.08149,1.08170,1.08067,1.08067,0.000398,1.081250,1.081122,2.539000e-07,1.081516,1.081263,2.674300e-07,1.081175,1.081478,4.298500e-07,1.081293,1.081642,4.250897e-07,1.082106,1.082037,1.609267e-06,-0.000746,0.000006,-0.000752,36.018788,34.603288,31.939255,34.647316,41.507568,-77.833753,1.080591,1.081516,1.082441,0.171071,0.042743,0.999233,16,3,-0.942261,-0.334880,1.224647e-16,-1.0,0.000185,2020-12-31 21:00:00,21:00:00,0.694193,0.5,100.000000,NaN
1,68363,2021-01-03 17:00:00,1.08074,1.08115,1.08010,1.08110,0.000666,1.081090,1.081111,1.723000e-07,1.081328,1.081208,1.978700e-07,1.081259,1.081409,3.296100e-07,1.081213,1.081565,3.614897e-07,1.081975,1.081947,1.509194e-06,-0.000729,0.000018,-0.000747,40.231568,40.594810,42.344204,33.326930,36.936379,-62.138728,1.080532,1.081328,1.082124,0.147176,0.356735,1.000398,17,6,-0.997669,-0.068242,-2.449294e-16,1.0,0.000185,2021-01-03 22:00:00,22:00:00,0.645258,0.5,100.000000,NaN
2,68364,2021-01-03 18:00:00,1.08124,1.08192,1.08090,1.08182,-0.000148,1.081197,1.081465,3.376333e-07,1.081334,1.081412,2.049800e-07,1.081379,1.081484,3.032100e-07,1.081210,1.081601,3.573000e-07,1.081873,1.081935,1.308399e-06,-0.000650,0.000077,-0.000728,46.574809,49.245953,55.892889,42.267945,36.747397,-33.223684,1.080524,1.081334,1.082144,0.149796,0.800038,1.000666,18,6,-0.979084,0.203456,-2.449294e-16,1.0,0.000185,2021-01-03 23:00:00,23:00:00,0.434800,0.5,100.000000,NaN
3,68365,2021-01-03 19:00:00,1.08181,1.08215,1.08161,1.08166,0.000841,1.081527,1.081563,1.429333e-07,1.081350,1.081495,2.161000e-07,1.081518,1.081516,1.538622e-07,1.081256,1.081610,3.699923e-07,1.081765,1.081909,1.101037e-06,-0.000594,0.000107,-0.000701,45.421170,47.536637,52.596920,60.425644,45.340173,-23.360656,1.080518,1.081350,1.082182,0.153803,0.686393,0.999852,19,6,-0.887885,0.460065,-2.449294e-16,1.0,0.000185,2021-01-04 00:00:00,00:00:00,0.486729,0.5,100.000000,NaN
4,68366,2021-01-03 20:00:00,1.08167,1.08273,1.08162,1.08257,0.000046,1.082017,1.082066,2.360333e-07,1.081564,1.081853,5.253300e-07,1.081612,1.081708,2.656178e-07,1.081434,1.081747,3.969423e-07,1.081699,1.081972,8.928937e-07,-0.000471,0.000184,-0.000655,52.610767,56.974199,66.200196,79.324494,60.672694,-5.442177,1.080267,1.081564,1.082861,0.239756,0.887951,1.000841,20,6,-0.730836,0.682553,-2.449294e-16,1.0,0.000185,2021-01-04 01:00:00,01:00:00,0.463180,0.5,100.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,74471,2021-12-24 11:00:00,1.03951,1.04020,1.03934,1.03934,0.000481,1.039650,1.039538,1.533000e-07,1.039708,1.039629,9.207000e-08,1.039830,1.039703,2.787333e-07,1.039682,1.039738,2.938141e-07,1.039675,1.039862,2.866682e-07,-0.000269,0.000057,-0.000327,44.044391,43.594124,40.953177,36.279070,43.100775,-78.139535,1.039165,1.039708,1.040251,0.104413,0.161012,0.999827,11,4,0.136167,-0.990686,-8.660254e-01,-0.5,0.000192,2021-12-24 16:00:00,16:00:00,0.532776,0.5,94.976567,None
6110,74472,2021-12-24 12:00:00,1.03934,1.04009,1.03900,1.03984,-0.000269,1.039567,1.039689,6.413334e-08,1.039744,1.039699,9.423000e-08,1.039896,1.039728,2.269600e-07,1.039716,1.039752,2.875256e-07,1.039673,1.039860,2.858871e-07,-0.000240,0.000069,-0.000309,48.571659,49.986673,52.466168,32.403101,38.036176,-54.883721,1.039195,1.039744,1.040293,0.105627,0.587412,1.000481,12,4,-0.136167,-0.990686,-8.660254e-01,-0.5,0.000192,2021-12-24 17:00:00,17:00:00,0.468823,0.5,94.976567,None
6111,74473,2021-12-24 13:00:00,1.0

In [16]:

hits = prediction.loc[((prediction['transaction'] == 'buy') & (prediction['y_pred'] > 0)) |
                      ((prediction['transaction'] == 'sell') & (prediction['y_pred'] < 0))].shape[0]
transactions = prediction.loc[prediction['transaction'].isin(['buy', 'sell'])].shape[0]

hits_ratio = hits / transactions
hits_ratio

0.5834076717216771

In [20]:
hours_to_exclude = [datetime.time(21,0,0), datetime.time(22,0,0),  datetime.time(23,0,0)]
excluded_hours_df = prediction.loc[~prediction['hour_local'].isin(hours_to_exclude)].copy()

correct = excluded_hours_df.loc[((excluded_hours_df['prediction'] > 0.55) & (excluded_hours_df['y_pred'] > 0)) |
               ((excluded_hours_df['prediction'] < 0.45) & (excluded_hours_df['y_pred'] < 0))].shape[0]

all_predictions = excluded_hours_df.loc[(excluded_hours_df['prediction'] > 0.55) | (excluded_hours_df['prediction'] < 0.45)].shape[0]

correct/all_predictions

0.5834076717216771

In [23]:
excluded_hours_df.loc[(excluded_hours_df['prediction'] > 0.55) | (excluded_hours_df['prediction'] < 0.45)].shape

(1121, 50)